下記のリンクをクリックするとGoogle Colabで実行することが出来ます  
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/2Nike2/LangChainPractice/blob/main/filename.ipynb)

### (事前準備: OpenAI APIキーの設定)
OpenAI APIを使う為のAPIキーを設定します  
このAPIキーについては、OpenAIのサイトで取得することが出来ます  
https://platform.openai.com/api-keys  
APIキーについては公開しないように注意してください  

In [ ]:
import os

# ここにあなたのOpenAIのAPIキーを入力してください
openai_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = openai_api_key


### Neo4jインストール
グラフ型データベースNeo4jを下記に従ってインストールする  
1. https://neo4j.com/docs/operations-manual/current/installation/   
   でOSを選択  
2. OSを選択して飛んだページにある作業を行う  
   a. Open JDK or Oracle Java 17のインストール(詳細は割愛)  
   b. https://neo4j.com/deployment-center/  
      「Graph Database Self-Managed」のところに移り、タブを「COMMUNITY」、OSを適宜変更  
      (Windowsならば「Windows Executable」、Macならば「Linux / Mac Executable」)、バージョンはデフォルト(最新版)で「Download」をクリック  
      その後、適切な場所(後々にファイル、フォルダの移動をすることがないような場所が良い)におく  


### Neo4j各種設定
下記の設定を行う
1. neo4j-community-x.x.x/labs/apoc-x.x.x-core.jar  
   を  
   neo4j-community-x.x.x/plugins/apoc-x.x.x-core.jar  
   に移動
2. (必要ならば)上記jarを実行できるように権限を変更する
   (Macだとchmodを使用、Windowsだとファイルのプロパティから「セキュリティ」を「許可する」か?)
3. neo4j-community-x.x.x/conf/neo4j.conf に下記の2行を追加または置き換え(同名の項目がすでにあるならばそこに近いところに記載すると良い)  
```
dbms.security.procedures.unrestricted=apoc.*  
dbms.security.procedures.allowlist=apoc.*,gds.*  
```
4. neo4j-community-x.x.x/bin/neo4j start  
   でneo4jを起動し、  
   Username:neo4j  
   Password:neo4j  
   でログイン  
   パスワード変更を要求されるので、変更後のパスワードを入力、変更後のパスワードは後に使うのでメモする  

5. 下記の環境変数を設定する
```
NEO4J_URI=bolt://localhost:7687
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=上記でメモした変更後のパスワード
```
(Windowsならば「環境変数の設定」、Macならば「.bash_profile」に記載して再起動、
またはdotenvを利用するならば「.env」ファイルを作成し、
```
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()
```
で読み込んでも良い)


## 知識グラフの活用
知識グラフを用いたQAチェインを作成します。

### ライブラリのインストール
LangChainのライブラリをインストールします

In [ ]:
!pip install langchain==0.1.4
!pip install langchain-openai==0.0.5
!pip install neo4j==5.18.0

### モデルの初期化
OpenAI APIを使う為のモデルを初期化します

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()


### グラフデータベースへの接続
neo4j-community-x.x.x/bin/neo4j start  
neo4j-community-x.x.x/bin/neo4j console
で起動されていることを確認  
(上記のパスワード変更から続けて行なっているならば基本的に必要がないが、もし止めていたら再び起動する)


In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()


### グラフの作成

In [ ]:
movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)


### グラフのスキーマの表示

In [ ]:
graph.refresh_schema()
print(graph.schema)


### グラフへの問い合わせ

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({'query': 'What was the cast of Casino?'})
response


In [ ]:
response = chain.invoke({'query': 'Tell me about jenres of Toy Story.'})
response


In [ ]:
response = chain.invoke({'query': 'What\'s the rate of Taxi Driver.'})
response


In [ ]:
chain_２ = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain_2.invoke({"query": "What was the cast of the Othello?"})
response
